In [ ]:
from tqdm import tqdm
import numpy as np

## Part 1

In [ ]:
#counts points for a matrix
def count_points(matrix):
    total_depth = len(matrix)
    points = 0
    for ydx, line in enumerate(matrix):
        for block in line:
            if block =='O':
                points += total_depth - ydx
    return points


test1 = np.array([['O', 'O', '#', '.'], # 5+5
                  ['.', '#', '.', '.'],
                  ['.', '.', '.', '#'],
                  ['O', 'O', 'O', 'O'], # 2+2+2+2
                  ['.', 'O', '#', 'O']]) #1+1
print(count_points(test1))

In [ ]:
# generates a string with same numbers of O and . but with all the O to the beginning
def shift_Os(array):
    n_Os = (array =='O').sum()
    n_dots = len(array) - n_Os
    return ['O']*n_Os +['.']*n_dots

# for each block between beginning, hashes and end, pushes all O towards the beginning
def push_col_north(col):
    col_length = len(col)
    pos_hash= np.append([-1] ,np.append(np.where(col == '#')[0], [col_length]))
    for first_hash, second_hash in zip(pos_hash[:-1], pos_hash[1:]):
        if 'O' in col[first_hash+1 : second_hash]:
            col[first_hash+1 : second_hash] = shift_Os(col[first_hash+1 : second_hash]) 
    return col

In [ ]:
total_map = []
# generate maps 
with open('Day14_input.txt') as f:
    for line in f:
        line = list(line.strip('\n'))
        total_map.append(line)
total_map = np.array(total_map)

n_cols = len(total_map[0])
# 1 push North for part 1
for ydx in range(n_cols):
        total_map[:,ydx] = push_col_north(total_map[:,ydx])
        
print(count_points(total_map))

## Part 2

In [ ]:
# does a NWSE cycle of pushing
def do_four_tilts(total_map, verbose = False):
    n_rows = len(total_map)
    n_cols = len(total_map[0])
    if verbose:
        print('\nOriginal\n', total_map, '\n')
    #North
    for ydx in range(n_cols):
        total_map[:,ydx] = push_col_north(total_map[:,ydx])
    if verbose:
        print('\nNorth\n', total_map, '\n')
    #West
    for xdx in range(n_rows):
        total_map[xdx,:] = push_col_north(total_map[xdx,:])                                                                
    if verbose:
        print('\nWest\n', total_map, '\n')                                                                 
    #South
    for ydx in range(n_cols):
        total_map[:,ydx] = np.flipud(push_col_north(np.flipud(total_map[:,ydx])))    
    if verbose:
        print('\nSouth\n', total_map, '\n')
    #East
    for xdx in range(n_rows):
        total_map[xdx,:] = np.flipud(push_col_north(np.flip(total_map[xdx,:])))
    if verbose:
        print('\nEast\n', total_map, '\n')        
    return total_map

test1 = np.array([['O', 'O', '#', '.'], 
                  ['.', '#', '.', '.'],
                  ['.', '.', '.', '#'],
                  ['O', 'O', 'O', 'O'],
                  ['.', 'O', '#', 'O']])



test1 = []
# generate maps 
with open('Day14_input.txt') as f:
    for line in f:
        line = list(line.strip('\n'))
        test1.append(line)

test1 = np.array(test1)
for spin in range(10):
    print('new cycle\n')
    test1 = do_four_tilts(test1, verbose = True)
    

In [ ]:
total_map = []
# generate maps 
with open('Day14_input.txt') as f:
    for line in f:
        line = list(line.strip('\n'))
        total_map.append(line)
total_map = np.array(total_map)


n_spins = pow(10,9)

#we choose a threshold where we hope the system has converged to a cycle
rounds_threshold = 400
for spin in tqdm(range(n_spins)):
    total_map = do_four_tilts(total_map)
        
    if spin == rounds_threshold: 
        test_map = np.copy(total_map)
    # if at the end of a cycle we are in the same position as we were during the threshold, we converged
    if spin> rounds_threshold and np.array_equal(total_map, test_map):
        print('iteration number ', spin, ' matches with iteration ', rounds_threshold)
        break

# compute how many iterations still needed
periodicity = spin - rounds_threshold
remaining_spins = (n_spins -1-spin)%periodicity

print('Executed ', spin , ' spins')
print('Found periodicity ', periodicity)
print('Remaining spins = ' ,remaining_spins )
# execute remaining cycles
for spin in tqdm(range(remaining_spins)):
    total_map = do_four_tilts(total_map)              


print(count_points(total_map))